In [2]:
from linear_algebra import Vector, dot

def sum_of_squares(v: Vector) -> float:
    """Computes the sum of squared elements in v"""
    return dot(v, v)

In [3]:
from typing import Callable

def difference_quotient(f: Callable[[float], float],
                        x: float,
                        h: float) -> float:
    return (f(x + h) - f(x)) / h

def square(x: float) -> float:
    return x * x

def derivative(x: float) -> float:
    return 2 * x

def estimate_gradient(f: Callable[[Vector], float],
                      v: Vector,
                      h: float = 0.0001):
    return [partial_difference_quotient(f, v, i, h)
            for i in range(len(v))]

In [5]:
import random
from linear_algebra import distance, add, scalar_multiply

def gradient_step(v: Vector, gradient: Vector, step_size: float) -> Vector:
    """Moves `step_size` in the `gradient` direction from `v`"""
    assert len(v) == len(gradient)
    step = scalar_multiply(step_size, gradient)
    return add(v, step)

def sum_of_squares_gradient(v: Vector) -> Vector:
    return [2 * v_i for v_i in v]

# x ranges from -50 to 49, y is always 20 * x + 5
inputs = [(x, 20 * x + 5) for x in range(-50, 50)]

def linear_gradient(x: float, y: float, theta: Vector) -> Vector:
    slope, intercept = theta
    predicted = slope * x + intercept    # The prediction of the model.
    error = (predicted - y)              # error is (predicted - actual)
    squared_error = error ** 2           # We'll minimize squared error
    grad = [2 * error * x, 2 * error]    # using its gradient.
    return grad

In [6]:
from typing import TypeVar, List, Iterator

T = TypeVar('T')  # this allows us to type "generic" functions

def minibatches(dataset: List[T],
                batch_size: int,
                shuffle: bool = True) -> Iterator[List[T]]:
    """Generates `batch_size`-sized minibatches from the dataset"""
    # Start indexes 0, batch_size, 2 * batch_size, ...
    batch_starts = [start for start in range(0, len(dataset), batch_size)]

    if shuffle: random.shuffle(batch_starts)  # shuffle the batches

    for start in batch_starts:
        end = start + batch_size
        yield dataset[start:end]


In [7]:
def main():
    xs = range(-10, 11)
    actuals = [derivative(x) for x in xs]
    estimates = [difference_quotient(square, x, h=0.001) for x in xs]
    
    # plot to show they're basically the same
    import matplotlib.pyplot as plt
    plt.title("Actual Derivatives vs. Estimates")
    plt.plot(xs, actuals, 'rx', label='Actual')       # red  x
    plt.plot(xs, estimates, 'b+', label='Estimate')   # blue +
    plt.legend(loc=9)
    # plt.show()
    
    
    plt.close()
    
    def partial_difference_quotient(f: Callable[[Vector], float],
                                    v: Vector,
                                    i: int,
                                    h: float) -> float:
        """Returns the i-th partial difference quotient of f at v"""
        w = [v_j + (h if j == i else 0)    # add h to just the ith element of v
             for j, v_j in enumerate(v)]
    
        return (f(w) - f(v)) / h
    
    
    # "Using the Gradient" example
    
    # pick a random starting point
    v = [random.uniform(-10, 10) for i in range(3)]
    
    for epoch in range(1000):
        grad = sum_of_squares_gradient(v)    # compute the gradient at v
        v = gradient_step(v, grad, -0.01)    # take a negative gradient step
        print(epoch, v)
    
    assert distance(v, [0, 0, 0]) < 0.001    # v should be close to 0
    
    
    # First "Using Gradient Descent to Fit Models" example
    
    from scratch.linear_algebra import vector_mean
    
    # Start with random values for slope and intercept.
    theta = [random.uniform(-1, 1), random.uniform(-1, 1)]
    
    learning_rate = 0.001
    
    for epoch in range(5000):
        # Compute the mean of the gradients
        grad = vector_mean([linear_gradient(x, y, theta) for x, y in inputs])
        # Take a step in that direction
        theta = gradient_step(theta, grad, -learning_rate)
        print(epoch, theta)
    
    slope, intercept = theta
    assert 19.9 < slope < 20.1,   "slope should be about 20"
    assert 4.9 < intercept < 5.1, "intercept should be about 5"
    
    
    # Minibatch gradient descent example
    
    theta = [random.uniform(-1, 1), random.uniform(-1, 1)]
    
    for epoch in range(1000):
        for batch in minibatches(inputs, batch_size=20):
            grad = vector_mean([linear_gradient(x, y, theta) for x, y in batch])
            theta = gradient_step(theta, grad, -learning_rate)
        print(epoch, theta)
    
    slope, intercept = theta
    assert 19.9 < slope < 20.1,   "slope should be about 20"
    assert 4.9 < intercept < 5.1, "intercept should be about 5"
    
    
    # Stochastic gradient descent example
    
    theta = [random.uniform(-1, 1), random.uniform(-1, 1)]
    
    for epoch in range(100):
        for x, y in inputs:
            grad = linear_gradient(x, y, theta)
            theta = gradient_step(theta, grad, -learning_rate)
        print(epoch, theta)
    
    slope, intercept = theta
    assert 19.9 < slope < 20.1,   "slope should be about 20"
    assert 4.9 < intercept < 5.1, "intercept should be about 5"
    
if __name__ == "__main__": main()

0 [-8.122748595031782, -8.685518502990579, -4.589555479788621]
1 [-7.960293623131146, -8.511808132930767, -4.497764370192849]
2 [-7.801087750668523, -8.341571970272152, -4.407809082788992]
3 [-7.645065995655152, -8.174740530866709, -4.319652901133212]
4 [-7.492164675742049, -8.011245720249374, -4.233259843110548]
5 [-7.342321382227208, -7.851020805844386, -4.148594646248337]
6 [-7.195474954582664, -7.694000389727498, -4.06562275332337]
7 [-7.051565455491011, -7.540120381932948, -3.984310298256903]
8 [-6.910534146381191, -7.389317974294289, -3.904624092291765]
9 [-6.772323463453567, -7.2415316148084035, -3.8265316104459295]
10 [-6.636876994184496, -7.096700982512235, -3.750000978237011]
11 [-6.504139454300805, -6.95476696286199, -3.6750009586722707]
12 [-6.374056665214789, -6.815671623604751, -3.601500939498825]
13 [-6.246575531910493, -6.679358191132656, -3.5294709207088486]
14 [-6.121644021272283, -6.545771027310003, -3.4588815022946715]
15 [-5.999211140846838, -6.414855606763803, -3.

886 [-1.3677479586289227e-07, -1.4625098958947401e-07, -7.728116985344789e-08]
887 [-1.340392999456344e-07, -1.4332596979768453e-07, -7.573554645637893e-08]
888 [-1.3135851394672172e-07, -1.4045945040173083e-07, -7.422083552725135e-08]
889 [-1.287313436677873e-07, -1.3765026139369621e-07, -7.273641881670633e-08]
890 [-1.2615671679443155e-07, -1.3489725616582229e-07, -7.12816904403722e-08]
891 [-1.2363358245854293e-07, -1.3219931104250583e-07, -6.985605663156475e-08]
892 [-1.2116091080937206e-07, -1.2955532482165572e-07, -6.845893549893346e-08]
893 [-1.1873769259318462e-07, -1.269642183252226e-07, -6.70897567889548e-08]
894 [-1.1636293874132093e-07, -1.2442493395871816e-07, -6.57479616531757e-08]
895 [-1.1403567996649451e-07, -1.219364352795438e-07, -6.443300242011218e-08]
896 [-1.1175496636716462e-07, -1.1949770657395293e-07, -6.314434237170994e-08]
897 [-1.0951986703982133e-07, -1.1710775244247387e-07, -6.188145552427573e-08]
898 [-1.073294696990249e-07, -1.147655973936244e-07, -6.064

ModuleNotFoundError: No module named 'scratch.linear_algebra'